## Download GEE: Elevation and slope
This script aims to download automatically Elevation and Slope data that match the images of the DRIVENDATA competition. Assuming that you have a google earth engine account, each image will be downloaded into a Google Drive folder of the same account.

In [1199]:
import os
from osgeo import gdal, ogr, osr
import ee
print(ee.__version__)
from time import sleep

0.1.278


In [1200]:
ee.Initialize()

Defining path where images are located

In [1234]:
path_root = r'D:\DOCTORATE\Competitions\Drivendata\data\final\public\train_labels'
folder_drive = 'Doctorado'

In [1237]:
Functions definition:

SyntaxError: invalid syntax (<ipython-input-1237-26bc9cfd8f68>, line 1)

In [1235]:
def get_polygon(path):
    #Getobject
    rast_data_source = gdal.Open(path)
    gt = rast_data_source.GetGeoTransform()
    #Raster info
    minx = gt[0]
    maxy = gt[3]
    maxx = minx + gt[1] * rast_data_source.RasterXSize
    miny = maxy + gt[5] * rast_data_source.RasterYSize
    wkt = f'POLYGON (({minx} {maxy},{maxx} {maxy},{maxx} {miny},{minx} {miny},{minx} {maxy}))'
    poly = ogr.CreateGeometryFromWkt(wkt)
    #spatial reference
    native_projection = osr.SpatialReference()
    native_projection.ImportFromWkt(rast_data_source.GetProjection())
    target = osr.SpatialReference()
    target.ImportFromEPSG(4326)
    transform = osr.CoordinateTransformation(native_projection, target)
    poly.Transform(transform)
    #get reprojected coordinates
    pt = []
    for fea in range(poly.GetGeometryRef(0).GetPointCount()):
        lat,long,h = poly.GetGeometryRef(0).GetPoint(fea)
        pt.append([long,lat])
    #Create an ee.Geometry
    polygon = ee.Geometry.Polygon(pt)
    #reference system number of source layer
    Epsg = native_projection.GetAttrValue("AUTHORITY", 0)+":" + native_projection.GetAttrValue("AUTHORITY", 1) 
    rast_data_source = None
    return polygon, Epsg
    
def get_gee_elevation(name_file,polygon, Epsg):
    #get DEM layer
    elevation = ee.Image("USGS/SRTMGL1_003").select('elevation').clip(polygon)
    elevation1 = elevation.resample('bilinear').reproject(crs=Epsg, scale=10)
    
    #save data
    task_config = {
        'folder': folder_drive,
        'fileFormat': 'GeoTIFF',
        'region' : polygon,
        'dimensions': 512
    }
    name_file_inf = name_file + "_dem"
    task = ee.batch.Export.image.toDrive(elevation1, name_file_inf, **task_config)
    task.start()
    k = 1
    #sleep loop until data is downloaded
    while task.status()['state'] == 'COMPLETED':
        sleep(5)
        k += 1
        print(f'Iteration Elevation: {k}')
    elevation = None
    task = None

def get_gee_slope(name_file,polygon, Epsg):
    #get Slope layer
    elevation = ee.Image("USGS/SRTMGL1_003").select('elevation').clip(polygon)
    slope = ee.Terrain.slope(elevation)
    slope1 = slope.resample('bilinear').reproject(crs=Epsg, scale=10)
    #save data
    task_config = {
        'folder': folder_drive,
        'fileFormat': 'GeoTIFF',
        'region' : polygon,
        'dimensions': 512
    }
    name_file_inf = name_file + "_slp"
    task = ee.batch.Export.image.toDrive(slope1, name_file_inf, **task_config)
    task.start()
    k = 1
    #sleep loop until data is downloaded
    while task.status()['state'] == 'COMPLETED':
        sleep(5)
        k += 1
        print(f'Iteration slope: {k}')
    slope1 = None
    task = None

Iteration through image files

In [1236]:
for i in os.listdir(path_root):
    if i.endswith('.tif'):
        path = os.path.join(path_root,i)
        polygon, Epsg = get_polygon(path)
        #get DEM
        name_file = i.split('.tif')[0]
        get_gee_elevation(name_file, polygon, Epsg)
        #get Slope
        get_gee_slope(name_file, polygon, Epsg)
        print(f'Done with image {name_file}')

Done with image awc00
Done with image awc01
Done with image awc02
Done with image awc03
Done with image awc04
Done with image awc05
Done with image awc06
Done with image awc07
Done with image awc08
Done with image awc09
Done with image awc10
Done with image awc11
Done with image awc12
Done with image awc13
Done with image awc14
Done with image ayt00
Done with image ayt01
Done with image ayt02
Done with image ayt03
Done with image ayt04
Done with image ayt05
Done with image ayt06
Done with image ayt07
Done with image ayt08
Done with image ayt09
Done with image ayt10
Done with image ayt11
Done with image ayt12
Done with image ayt13
Done with image ayt14
Done with image ayt15
Done with image ayt16
Done with image ayt17
Done with image coz00
Done with image coz01
Done with image coz02
Done with image coz03
Done with image coz04
Done with image coz05
Done with image coz06
Done with image coz07
Done with image coz08
Done with image coz09
Done with image coz10
Done with image coz11
Done with 

Done with image qus18
Done with image qus19
Done with image qus20
Done with image qus21
Done with image qus22
Done with image qus23
Done with image qus24
Done with image qus25
Done with image qus26
Done with image qus27
Done with image qus28
Done with image qus29
Done with image qxb00
Done with image qxb01
Done with image qxb02
Done with image qxb03
Done with image qxb04
Done with image qxb05
Done with image qxb06
Done with image qxb07
Done with image qxb08
Done with image qxb09
Done with image qxb10
Done with image qxb11
Done with image qxb12
Done with image qxb13
Done with image qxb14
Done with image qxb15
Done with image qxb16
Done with image qxb17
Done with image qxb18
Done with image qxb19
Done with image qxb20
Done with image qxb21
Done with image qxb22
Done with image qxb23
Done with image qxb24
Done with image qxb25
Done with image qxb26
Done with image qxb27
Done with image qxb28
Done with image qxb29
Done with image qxb30
Done with image qxb31
Done with image qxb32
Done with 